In [ ]:
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image, Table, TableStyle, PageBreak
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.pagesizes import A4
from reportlab.lib import colors
from reportlab.lib.units import cm
import Config

# ----------------------------
# CONFIG
# ----------------------------
PERSON_FOLDERS_DIR = r'{}\Working environment'.format(Config.BASE_PATH)
NUM_PERSONS = 36
OUTPUT_FILENAME_TEMPLATE = "Person_{pid}_Master_Report.pdf"

# Expected columns in gold files
EXPECTED_COLUMNS = [
    "transaction_id", "area_id", "service_type", "transaction_amount", "transaction_date",
    "record_id", "units_processed", "processing_cost", "operation_date"
]

# Explanation text (same for all reports). Tone: analytical corporate.
EXPLANATION_TEXT = """
Executive Summary:
This report provides an analytical summary of key performance indicators (KPIs) by service.
KPIs include transaction counts, transaction amounts, processing units, and processing costs.
Percentages indicate each service's contribution to the overall totals for the person’s assigned areas.
Charts visualize the distribution of transaction amounts and operational costs across services, helping stakeholders quickly identify high-value services and cost concentration. Use these insights for resource allocation, cost-control measures, and performance improvement initiatives.
"""

# small helper to format numbers
def fmt(x):
    if pd.isna(x):
        return "-"
    if isinstance(x, (int,)) :
        return f"{x:,}"
    try:
        return f"{x:,.2f}"
    except Exception:
        return str(x)

# ----------------------------
# Function to generate KPIs and PDF for a single person folder
# ----------------------------
def generate_person_report(person_id):
    person_folder = os.path.join(PERSON_FOLDERS_DIR, f"Person_{person_id}")
    gold_folder = os.path.join(person_folder, "Gold")
    os.makedirs(gold_folder, exist_ok=True)

    # collect all Service_*.xlsx files in Gold
    service_files = glob.glob(os.path.join(gold_folder, "Service_*.xlsx"))
    service_files.sort()

    output_pdf_path = os.path.join(gold_folder, OUTPUT_FILENAME_TEMPLATE.format(pid=person_id))

    # If no files present, create a PDF with a brief message and exit
    if not service_files:
        doc = SimpleDocTemplate(output_pdf_path, pagesize=A4)
        styles = getSampleStyleSheet()
        story = []
        story.append(Paragraph(f"Person {person_id} - Master KPI Report", styles['Title']))
        story.append(Spacer(1, 12))
        story.append(Paragraph("No service files found in Gold folder.", styles['Normal']))
        story.append(Spacer(1, 12))
        story.append(Paragraph("Please ensure Service_*.xlsx files are present under the Gold folder before running this report generation.", styles['Normal']))
        doc.build(story)
        print(f"[Person {person_id}] No service files found. Report created with message: {output_pdf_path}")
        return

    # Load and concatenate all service files into a single DataFrame, track service source
    frames = []
    for fpath in service_files:
        try:
            df = pd.read_excel(fpath)
        except Exception as e:
            print(f"[Person {person_id}] Error reading {fpath}: {e}. Skipping.")
            continue

        # Ensure expected columns exist - if missing, add as NaN to avoid crashes
        for col in EXPECTED_COLUMNS:
            if col not in df.columns:
                df[col] = pd.NA

        # Guarantee correct service_type column exists: infer from filename if absent or uniform
        if 'service_type' not in df.columns or df['service_type'].isna().all():
            service_name = os.path.splitext(os.path.basename(fpath))[0]  # e.g., Service_1
            df['service_type'] = service_name

        frames.append(df[EXPECTED_COLUMNS])

    if not frames:
        doc = SimpleDocTemplate(output_pdf_path, pagesize=A4)
        styles = getSampleStyleSheet()
        story = [Paragraph(f"Person {person_id} - Master KPI Report", styles['Title']),
                 Spacer(1, 12),
                 Paragraph("No readable service data was found. Check file formats.", styles['Normal'])]
        doc.build(story)
        print(f"[Person {person_id}] No readable data. PDF created: {output_pdf_path}")
        return

    all_df = pd.concat(frames, ignore_index=True)

    # Convert numeric columns if needed
    all_df['transaction_amount'] = pd.to_numeric(all_df['transaction_amount'], errors='coerce').fillna(0)
    all_df['units_processed'] = pd.to_numeric(all_df['units_processed'], errors='coerce').fillna(0)
    all_df['processing_cost'] = pd.to_numeric(all_df['processing_cost'], errors='coerce').fillna(0)

    # Group by service_type
    grouped = all_df.groupby('service_type', dropna=False)

   # KPI calculations
    kpi_df = pd.DataFrame({
        "service_type": grouped.size().index,
        "num_transactions": grouped['transaction_id'].nunique().values,  # unique transactions
        "total_transaction_amount": grouped['transaction_amount'].sum().values,
        "avg_transaction_amount": grouped['transaction_amount'].mean().values,
        "total_units_processed": grouped['units_processed'].sum().values,
        "total_processing_cost": grouped['processing_cost'].sum().values
    })

    # Compute avg processing cost per unit safely to avoid division by zero
    kpi_df["avg_processing_cost_per_unit"] = np.where(
        kpi_df["total_units_processed"] == 0,
        0,
        kpi_df["total_processing_cost"] / kpi_df["total_units_processed"]
    )
    # Overall totals for percentages
    total_tx_amount = kpi_df['total_transaction_amount'].sum()
    total_proc_cost = kpi_df['total_processing_cost'].sum()
    total_units = kpi_df['total_units_processed'].sum()
    total_transactions = kpi_df['num_transactions'].sum()

    # Add percentage columns (avoid divide-by-zero)
    if total_tx_amount == 0:
        kpi_df['transaction_amount_pct'] = 0.0
    else:
        kpi_df['transaction_amount_pct'] = (kpi_df['total_transaction_amount'] / total_tx_amount) * 100

    if total_proc_cost == 0:
        kpi_df['processing_cost_pct'] = 0.0
    else:
        kpi_df['processing_cost_pct'] = (kpi_df['total_processing_cost'] / total_proc_cost) * 100

    if total_units == 0:
        kpi_df['units_processed_pct'] = 0.0
    else:
        kpi_df['units_processed_pct'] = (kpi_df['total_units_processed'] / total_units) * 100

    # Sort services by transaction amount desc for nicer visuals
    kpi_df = kpi_df.sort_values(by='total_transaction_amount', ascending=False).reset_index(drop=True)

    # ----------------------------
    # Generate Charts (saved as temporary PNGs)
    # ----------------------------
    tmp_dir = os.path.join(gold_folder, "tmp_charts")
    os.makedirs(tmp_dir, exist_ok=True)

    chart_paths = {}

    # 1) Transaction Amount by Service - Pie Chart
    try:
        plt.figure(figsize=(6,6))
        labels = kpi_df['service_type'].astype(str)
        sizes = kpi_df['total_transaction_amount']
        # if too many slices, matplotlib will handle, labels will overlap; that's acceptable for now
        plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=140)
        plt.title("Transaction Amount by Service")
        tx_pie_path = os.path.join(tmp_dir, f"person_{person_id}_tx_amount_pie.png")
        plt.tight_layout()
        plt.savefig(tx_pie_path, dpi=150)
        plt.close()
        chart_paths['tx_pie'] = tx_pie_path
    except Exception as e:
        print(f"[Person {person_id}] Error creating tx pie chart: {e}")

    # 2) Processing Cost by Service - Bar Chart
    try:
        plt.figure(figsize=(8,4.5))
        plt.bar(kpi_df['service_type'].astype(str), kpi_df['total_processing_cost'])
        plt.xticks(rotation=45, ha='right')
        plt.title("Processing Cost by Service")
        plt.xlabel("Service")
        plt.ylabel("Total Processing Cost")
        cost_bar_path = os.path.join(tmp_dir, f"person_{person_id}_proc_cost_bar.png")
        plt.tight_layout()
        plt.savefig(cost_bar_path, dpi=150)
        plt.close()
        chart_paths['cost_bar'] = cost_bar_path
    except Exception as e:
        print(f"[Person {person_id}] Error creating cost bar chart: {e}")

    # 3) Units Processed by Service - Pie Chart
    try:
        plt.figure(figsize=(6,6))
        plt.pie(kpi_df['total_units_processed'], labels=kpi_df['service_type'].astype(str), autopct='%1.1f%%', startangle=140)
        plt.title("Units Processed by Service")
        units_pie_path = os.path.join(tmp_dir, f"person_{person_id}_units_pie.png")
        plt.tight_layout()
        plt.savefig(units_pie_path, dpi=150)
        plt.close()
        chart_paths['units_pie'] = units_pie_path
    except Exception as e:
        print(f"[Person {person_id}] Error creating units pie chart: {e}")

    # 4) Number of Transactions by Service - Bar Chart
    try:
        plt.figure(figsize=(8,4.5))
        plt.bar(kpi_df['service_type'].astype(str), kpi_df['num_transactions'])
        plt.xticks(rotation=45, ha='right')
        plt.title("Number of Transactions by Service")
        plt.xlabel("Service")
        plt.ylabel("Number of Transactions")
        txcount_bar_path = os.path.join(tmp_dir, f"person_{person_id}_txcount_bar.png")
        plt.tight_layout()
        plt.savefig(txcount_bar_path, dpi=150)
        plt.close()
        chart_paths['txcount_bar'] = txcount_bar_path
    except Exception as e:
        print(f"[Person {person_id}] Error creating txcount bar chart: {e}")

    # ---------------------------- ---------------------------- ---------------------------- ---------------------------- ----------------------------
    # Build PDF with reportlab
    # ---------------------------- ---------------------------- ---------------------------- ---------------------------- ----------------------------
    doc = SimpleDocTemplate(output_pdf_path, pagesize=A4, rightMargin=2*cm, leftMargin=2*cm)
    styles = getSampleStyleSheet()
    story = []

    # Title
    story.append(Paragraph(f"Person {person_id} - Master KPI Report", styles['Title']))
    story.append(Spacer(1, 12))

    # Explanation
    story.append(Paragraph(EXPLANATION_TEXT.strip(), styles['Normal']))
    story.append(Spacer(1, 12))
#####################################################################################################################    
    # ----------------------------
    # KPI explanation paragraph to add at the start of the PDF
    # ----------------------------
    KPI_EXPLANATION_TEXT = """
    <b>KPI Overview:</b> This report contains key performance indicators (KPIs) for all services assigned to this person.<br/><br/>
    The metrics are divided into two main categories:<br/><br/>

    <b>1. Volume Metrics (Table 1):</b><br/>
    - <b>Transactions:</b> Number of unique transactions per service.<br/>
    - <b>Transaction Amount:</b> Total monetary value of transactions per service.<br/>
    - <b>Transaction Amount (%):</b> Share of each service's transaction amount relative to the total.<br/>
    - <b>Units:</b> Total units processed per service.<br/>
    - <b>Units (%):</b> Share of each service's units relative to the total.<br/><br/>

    <b>2. Financial & Efficiency Metrics (Table 2):</b><br/>
    - <b>Processing Cost:</b> Total operational cost incurred per service.<br/>
    - <b>Processing Cost (%):</b> Share of each service's cost relative to total cost.<br/>
    - <b>Average Transaction Amount:</b> Mean transaction value per service.<br/>
    - <b>Average Cost per Unit:</b> Cost efficiency per unit processed, calculated as Processing Cost divided by Units.<br/><br/>

    <b>Verification Tips:</b><br/>
    - Percentages in the tables should sum to approximately 100%.<br/>
    - Totals of transactions, units, amounts, and costs should match underlying data.<br/>
    - Averages are derived from these totals for each service.<br/><br/>
    This explanation ensures consistent interpretation of the KPIs and supports data-driven decision-making.
    """

    # Convert to a reportlab Paragraph and add to story
    from reportlab.lib.styles import ParagraphStyle
    from reportlab.lib.enums import TA_JUSTIFY

    kpi_explanation_style = ParagraphStyle(
        name='KPIExplanation',
        fontSize=10,
        leading=14,
        alignment=TA_JUSTIFY,
        spaceAfter=12
    )

    # Append to the story after the title
    story.append(Paragraph(KPI_EXPLANATION_TEXT.strip(), kpi_explanation_style))
    story.append(Spacer(1, 12))
#####################################################################################################################

    # Summary numbers
    summary_table_data = [
        ["Metric", "Value"],
        ["Total Transaction Amount", fmt(total_tx_amount)],
        ["Total Processing Cost", fmt(total_proc_cost)],
        ["Total Units Processed", fmt(total_units)],
        ["Total Transactions (unique)", fmt(total_transactions)],
        ["Number of Services in Report", str(len(kpi_df))]
    ]
    summary_table = Table(summary_table_data, colWidths=[8*cm, 6*cm])
    summary_table.setStyle(TableStyle([
        ('BACKGROUND', (0,0), (-1,0), colors.HexColor("#d3d3d3")),
        ('ALIGN',(0,0),(-1,-1),'LEFT'),
        ('GRID', (0,0), (-1,-1), 0.25, colors.grey),
        ('FONTNAME', (0,0), (-1,0), 'Helvetica-Bold')
    ]))
    story.append(summary_table)
    story.append(Spacer(1, 12))

##########################################################################################################################################
    # Table 1 columns
    kpi_table1_header = ["Service", "Transactions", "Txn Amount", "Txn Amount (%)", "Units", "Units (%)"]
    # Table 2 columns
    kpi_table2_header = ["Service", "Proc Cost", "Proc Cost (%)", "Avg Txn Amt", "Avg Cost/Unit"]

    # Prepare rows for table 1
    kpi_table1_rows = [kpi_table1_header]
    for _, row in kpi_df.iterrows():
        kpi_table1_rows.append([
            str(row['service_type']),
            fmt(row['num_transactions']),
            fmt(row['total_transaction_amount']),
            f"{row['transaction_amount_pct']:.2f}%",
            fmt(row['total_units_processed']),
            f"{row['units_processed_pct']:.2f}%"
        ])

    # Prepare rows for table 2
    kpi_table2_rows = [kpi_table2_header]
    for _, row in kpi_df.iterrows():
        kpi_table2_rows.append([
            str(row['service_type']),
            fmt(row['total_processing_cost']),
            f"{row['processing_cost_pct']:.2f}%",
            fmt(row['avg_transaction_amount']),
            fmt(row['avg_processing_cost_per_unit'])
        ])

    # Build Table 1
    kpi_table1 = Table(kpi_table1_rows, colWidths=[2*cm, 3*cm, 2.5*cm, 3*cm, 2*cm, 2*cm])
    kpi_table1.setStyle(TableStyle([
        ('BACKGROUND', (0,0), (-1,0), colors.HexColor("#f2f2f2")),
        ('GRID', (0,0), (-1,-1), 0.25, colors.grey),
        ('FONTNAME', (0,0), (-1,0), 'Helvetica-Bold'),
        ('ALIGN',(1,1),(-1,-1),'RIGHT'),
        ('VALIGN',(0,0),(-1,-1),'MIDDLE')
    ]))

    # Build Table 2
    kpi_table2 = Table(kpi_table2_rows, colWidths=[2.5*cm, 2.5*cm, 3*cm, 3*cm, 3*cm])
    kpi_table2.setStyle(TableStyle([
        ('BACKGROUND', (0,0), (-1,0), colors.HexColor("#f2f2f2")),
        ('GRID', (0,0), (-1,-1), 0.25, colors.grey),
        ('FONTNAME', (0,0), (-1,0), 'Helvetica-Bold'),
        ('ALIGN',(1,1),(-1,-1),'RIGHT'),
        ('VALIGN',(0,0),(-1,-1),'MIDDLE')
    ]))

    # Append tables to story with spacing
    story.append(Spacer(1, 12))
    story.append(kpi_table1)
    story.append(Spacer(1, 12))
    story.append(kpi_table2)
    story.append(Spacer(1, 18))

##########################################################################################################################################

    # Charts: embed them one by one
    chart_order = ['tx_pie', 'cost_bar', 'units_pie', 'txcount_bar']
    chart_titles = {
        'tx_pie': "Transaction Amount Distribution by Service",
        'cost_bar': "Processing Cost by Service",
        'units_pie': "Units Processed Distribution by Service",
        'txcount_bar': "Number of Transactions by Service"
    }

    for key in chart_order:
        if key in chart_paths:
            # Add title
            story.append(Paragraph(chart_titles.get(key, ""), styles['Heading3']))
            story.append(Spacer(1,6))
            img_path = chart_paths[key]
            # scale image to page width while preserving aspect ratio
            img = Image(img_path, width=16*cm, height=9*cm)
            story.append(img)
            story.append(Spacer(1,12))

    # Footer / generation note
    story.append(Spacer(1, 12))
    story.append(Paragraph("Report generated automatically. Data reflects combined Gold outputs for assigned areas.", styles['Italic']))

    doc.build(story)

    # Clean up temporary charts
    try:
        for p in chart_paths.values():
            if os.path.exists(p):
                os.remove(p)
        if os.path.exists(tmp_dir) and not os.listdir(tmp_dir):
            os.rmdir(tmp_dir)
    except Exception as e:
        print(f"[Person {person_id}] Warning cleaning temp charts: {e}")

    print(f"[Person {person_id}] Report created: {output_pdf_path}")


# ----------------------------
# RUN for all persons
# ----------------------------
if __name__ == "__main__":
    for pid in range(1, NUM_PERSONS + 1):
        try:
            generate_person_report(pid)
        except Exception as e:
            print(f"[Person {pid}] ERROR: {e}")
# ---------------------------- ---------------------------- ---------------------------- ---------------------------- ----------------------------
# Build PDF with reportlab
# ---------------------------- ---------------------------- ---------------------------- ---------------------------- ----------------------------
